# 03. 데이터 전 처리하기

In [29]:
from pexpect import split_command_line
from sklearn import show_versions
show_versions()


System:
    python: 3.13.2 (v3.13.2:4f8bb3947cf, Feb  4 2025, 11:51:10) [Clang 15.0.0 (clang-1500.3.9.4)]
executable: /usr/local/bin/python3.13
   machine: macOS-26.0-arm64-arm-64bit-Mach-O

Python dependencies:
      sklearn: 1.6.1
          pip: 24.3.1
   setuptools: 75.8.0
        numpy: 1.26.4
        scipy: 1.16.0
       Cython: None
       pandas: 2.2.3
   matplotlib: 3.10.0
       joblib: 1.4.2
threadpoolctl: 3.5.0

Built with OpenMP: True

threadpoolctl info:
       user_api: openmp
   internal_api: openmp
    num_threads: 14
         prefix: libomp
       filepath: /Users/ihanjo/Library/Python/3.13/lib/python/site-packages/sklearn/.dylibs/libomp.dylib
        version: None


### 데이터의 특징 이해

ML알고리즘은 데이터 기반이므로 어떤 데이터를 입력하느냐에 따라 결과도 달라질 수 있다.

**[Garbage In, Garbage Out]**

- 결손값 NaN, Null 값은 고정된 다른 값으로 변환해야 한다.
- 사이킷런 알고리즘은 문자열 값을 입력으로 사용하지 않는다.
    - 숫자로 인코딩 되야 한다.
- 문자는 카테고리형 피처와 텍스트형 피처가 있을 수 있다.
    - 카테고리형은 코드 값으로
    - 텍스트형은 피처 벡터화 등 기법


데이터 세트에는 Gender, Married, Dependents, Self_Employed 과 Education과 같은 서로 다른 특징을 가질 수 있다. 이러한 모든 범주형 특성은 문자열 값이지만, 실제로 일어나는 일은 Logistic Regression, kNN 같은 거리 기반 방법, sklearn에서 SVM, Tree 기반 방법 등은 학습자들이 숫자 배열을 필요로 한다는 것입니다. 문자열 값이 있는 기능은 이러한 학습자가 처리할 수 없다.

feature engineering


### 데이터 전 처리의 역할
1. 머신러닝의 입력 형태로 **데이터 변환 (특성 엔지니어링)**
    - 대부분의 머신러닝 모델은 숫자 데이터를, 일반적으로 **행렬 형태**로 입력받는다.
    - 그러므로 전 처리를 통해 일반 데이터를 머신러닝 모델이 이해할 수 있는 **수치형 자료**로 변환해 주어야 한다.
2. 결측값 및 이상치를 처리하여 **데이터 정제**
    - NaN과 outlier 제거
3. 학습용 및 평가용 **데이터 분리**
    - 원본 데이터를 학습용 데이터와 평가용 데이터로 분리
    - 원본 데이터 전체를 학습용 데이터로 사용할 경우, 과적합 문제가 발생할 수 있다.
    - training(test) + validation



# 1. 범주형 자료 전 처리

### 대표적인 범주형의 One-Hot 인코딩


- 명목형 자료
  - 원-핫 인코딩으로 범주형을 변환
  - **수치 맵핑 방식**
    - 범주가 2개인 경우, 범주를 0, 1로 맵핑 `예) male 0, female 1`
    - 범주가 3개 이상인 경우, 수치의 크기 간격을 같게 하여 수치 맵핑 `예) S는 0, Q는 1, C는 2 (Embarked)`
  - **더미(Dummy) 기법**
    - 가변수(Dummy variable)를 사용해 각 범주를 0 또는 1로 변환
    - `예) Sex를 Sex_female과 Sex_male의 범주로 나누면, 남자는 Sex_female이 0, Sex_male이 1`
    - `Embarked를 Embarked_S, Embarked_Q, Embarked_C로 나누면, Q는 Embarked_S가 0, Embarked_Q가 1, Embarked_C가 0`

- 순서형 자료
  - 수치 맵핑 방식
    - 수치에 맵핑하여 변환하지만, 수치 간 크기 차이는 커스텀 가능
      - `예) 없음은 0, 조금 많음은 4, 매우 많음은 10`
    - 크기 차이가 머신러닝 결과에 영향을 끼칠 수 있음

### 명목형 자료 변환하기 - 수치 맵핑

명목형 단어를 대응하는 숫자로 변환한다.

예) setona -> 0, A학점 -> 95~98

In [54]:
import pandas as pd
# 데이터 읽기
titanic = pd.read_csv('./data/titanic.csv')
print('변환 전: \n',titanic['Sex'].head())


변환 전: 
 0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object


In [55]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [56]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [57]:
titanic['Sex'].unique()

array(['male', 'female'], dtype=object)

In [58]:
titanic['Age'].unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  ])

In [59]:
titanic['Pclass'].unique()

array([3, 1, 2])

In [60]:
# 1. replace를 사용하여 male -> 0, female -> 1로 변환합니다.
titanic['Sex'].replace({'male':0, 'female':1})

# 변환한 성별 데이터를 출력합니다.
titanic['Sex']

/var/folders/x_/xf7w4xhx6kl03__yxdgdfl9m0000gn/T/ipykernel_79069/530945398.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  titanic['Sex'].replace({'male':0, 'female':1})


0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: Sex, Length: 891, dtype: object

### 명목형 자료 변환하기 - 더미 방식

In [61]:
# Embarked 컬럼
titanic['Embarked'].unique()


array(['S', 'C', 'Q', nan], dtype=object)

In [62]:
# get_dummies를 사용하여 변환합니다.
dummies = pd.get_dummies(titanic['Embarked'])

# 변환한 Embarked 데이터를 출력합니다.
print('\n변환 후: \n',dummies.head())


변환 후: 
        C      Q      S
0  False  False   True
1   True  False  False
2  False  False   True
3  False  False   True
4  False  False   True


다른 컬럼을 get_dummies 로 변환

In [63]:
pd.get_dummies(titanic[['Embarked']])

,Embarked_C,Embarked_Q,Embarked_S
0,False,False,True
1,True,False,False
2,False,False,True
3,False,False,True
4,False,False,True
...,...,...,...
886,False,False,True
887,False,False,True
888,False,False,True
889,True,False,False


In [76]:
pd.get_dummies(titanic[['Sex']])

,Sex_female,Sex_male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True
...,...,...
886,False,True
887,True,False
888,True,False
889,False,True


### "실습" bmi2000.csv 명목형 전처리

In [74]:
# bmi20000.csv

df = pd.read_csv('./data/bmi20000.csv')

df.head(3)

,height,weight,label
0,125,43,normal
1,121,60,normal
2,183,57,thin


In [72]:
# 1. 숫자 맵핑
df['label'] = df['label'].replace({'thin':0, 'normal':2, 'fat':3})

/var/folders/x_/xf7w4xhx6kl03__yxdgdfl9m0000gn/T/ipykernel_79069/3806641235.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace({'thin':0, 'normal':2, 'fat':3})


In [75]:
# 2. one-hot encoding get_dummies 활용
pd.get_dummies(df['label'])

,fat,normal,thin
0,False,True,False
1,False,True,False
2,False,False,True
3,False,True,False
4,False,False,True
...,...,...,...
19995,True,False,False
19996,True,False,False
19997,False,True,False
19998,True,False,False


# 2. 수치형 자료 전 처리

- **수치형 자료** : 크기를 갖는 수치형 값으로 이루어진 데이터
- 머신러닝의 입력으로 바로 사용할 수 있으나, 모델의 성능을 높이기 위해 데이터 변환이 필요


### 대표적인 수치형 자료의 스케일링

수치형 자료의 경우 다른 수치형 자료와 범위를 맞추기 위해 스케일링을 통한 정규화 또는 표준화를 수행한다. **스케일링(Scaling)**은 변수 값의 범위 및 크기를 일정 범위로 변환하는 방식으로 변수(feature) 간의 범위가 차이가 나면 사용한다.  min-max scale, 정규화 스케일, 표준화 스케일링 등이 있다.

- **정규화(Normalization)**
  - **최소-최대 정규화(Min-Max Normalization)**
  - $X^\prime = \frac{X - X_{min}}{X_{max}-X_{min}}$<br>
  - 데이터를 모두 최소 0, 최대 1의 값으로 변환하는 것 (서로 다른 Feature의 scale를 통일하기 위해 변환)

- **표준화(Standardization)**
  - **Z-Score Scaling**
    - $X^\prime = \frac{x-\bar{x}}{\sigma}$
    - 데이터를 평균이 0, 분산이 1인 가우시안 정규분포로 만들어 주는 것



### Min-Max scale

모든 특성이 0와 1 사이에 위치하도록 데이터를 변경한다. 2차원 데이터  경우 모든 데이터가 x축의 0과 1사이에 사각 영역에 담기게 된다.

$$X^\prime = \frac{X - X_{min}}{X_{max}-X_{min}}
$$




In [79]:
import pandas as pd

# min-max scaling
def minmax_scale(data):
    return (data - data.min()) / (data.max() - data.min())

In [84]:
# 데이터를 읽어 옵니다.
print('변환 전: \n',titanic['Fare'].head())

변환 전: 
 0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64


정규화 작업을 하면 데이터의 범위가 0~1 사이로 변환된다.

In [86]:
titanic['Fare'].min(), titanic['Fare'].max()

(0.0, 512.3292)

In [87]:
# minmax 함수를 사용하여 정규화합니다.
df = minmax_scale(titanic['Fare'])

df.min(), df.max()

(0.0, 1.0)

In [88]:
# 변환한 Fare 데이터를 출력합니다.
df.head()

0    0.014151
1    0.139136
2    0.015469
3    0.103644
4    0.015713
Name: Fare, dtype: float64

Sklearn은 [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)를 통해 피처의 특성/속성이 다른 데이터를 0에서 1 사이로 축소할 수 있는 도구를 제공합니다. 

 - ML알고리즘에서 사용할 데이터 분포가 가우시안 분포가 아닌 경우 최소/최대 값 범위를 다루도록 할 수 있다.

In [89]:
from sklearn.preprocessing import MinMaxScaler

In [95]:
scaler = MinMaxScaler() # scaler model

In [96]:
scaler.fit(titanic[['Fare']])

MinMaxScaler()

### 수치형 자료 변환하기 - 표준화

**Z-Score Scaling** 라고도 하며  데이터를 평균이 0, 분산이 1인 가우시안 정규분포로 만들어 주는 것

$$X^\prime = \frac{x-\bar{x}}{\sigma}$$

In [97]:
# 표준화를 수행하는 함수를 구현합니다.
def standard(data):
    return (data - data.mean()) / data.std()

# 데이터를 읽어 옵니다.

print('변환 전: \n',titanic['Fare'].head())

변환 전: 
 0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64


In [98]:
# standard 함수를 사용하여 표준화합니다.
standard_data = standard(titanic['Fare'])

# 변환한 Fare 데이터를 출력합니다.
standard_data.head()

0   -0.502163
1    0.786404
2   -0.488580
3    0.420494
4   -0.486064
Name: Fare, dtype: float64

# 3. 데이터 정제 및 분리하기


## 데이터 Cleansing

#### 결측값(Missing data) 처리하기

값이 없는 경우

- 일반적인 머신러닝 모델의 입력 값으로 결측값을 사용할 수 없음
- 따라서 **Null, None, NaN** 등의 결측값을 처리해야 함
- 이것이 곧 '데이터 정제'의 과정임
- 대표적인 결측값 처리 방식
  1. 결측값이 존재하는 **샘플 삭제**
  2. 결측값이 많이 존재하는 **변수 삭제**
  3. 결측값을 **다른 값으로 대체**



결측값이 있는 데이터는 일반적으로 머신러닝의 입력으로 사용할 수 없습니다. 그렇기에 데이터 전 처리 과정에서는 삭제 또는 대체 방식으로 결측값을 처리합니다.
- **`drop(columns = ['feature'])`** : pandas의 DataFrame에서 특정 변수(columns)를 삭제하기 위해 사용
- **`dropna()`** : DataFrame에서 결측값이 있는 샘플을 제거하기 위해서 사용 (NaN 제거)



In [47]:
# 데이터를 읽어 옵니다.

# info


In [48]:
# 각 피처(컬럼)의 NaN 현황



In [49]:
# 1. Cabin 변수를 제거합니다.

# Cabin 변수를 제거 후 결측값이 어디에 남아 있는지 확인합니다.




In [50]:
# 2. 결측값이 존재하는 샘플 제거합니다.


# 결측값이 존재하는지 확인합니다.
print('결측값이 존재하는 샘플 제거')



결측값이 존재하는 샘플 제거


#### 이상치(Outlier) 처리하기

이상치/극단치는 데이터의 표준편차 범위 보다 아주 작은 값이나 아주 큰 값
- 이상치가 있으면, 모델의 성능을 저하할 수 있음
- 이상치는 일반적으로 전 처리 과정에서 제거하며, 어떤 값이 이상치인지 판단하는 기준이 중요함
- 이상치 판단 기준 방법
  1. 통계 지표(카이제곱 검정, IQR 지표 등)를 사용하여 판단
  2. 데이터 분포를 보고 직접 판단
  3. 머신러닝 기법을 사용하여 이상치 분류




이상치가 존재하는 데이터를 머신러닝에 사용하게 된다면 성능 저하를 야기할 수 있습니다. 그렇기에 데이터 전 처리 과정에서는 이상치를 판별하고 처리합니다.



결측값이 제거된 상태:

In [51]:
#데이터를 읽어 옵니다.
titanic = pd.read_csv('./data/titanic.csv')

# (Age 값 - 내림 Age 값)이 0보다 크다면 소수점을 갖는 데이터로 분류합니다.




In [52]:
# 이상치를 처리합니다.



## 지도학습 데이터 분리


#### 데이터 분리가 필요한 이유
- 머신러닝 모델을 평가하기 위해서는 학습에 사용하지 않은 평가용 데이터가 필요
- 약 7:3 ~ 8:2 비율로 학습용, 평가용 데이터를 분리함



지도학습의 경우 feature 데이터와 label 데이터를 분리하여 저장

- **Feature 데이터** : label을 예측하기 위한 입력 값

- **Label 데이터** : 예측해야 할 대상이 되는 데이터
  - `예) '공부 시간(feature)'에 따른 '시험 점수(label)'`
  - `예) titanic 데이터에서 feature data는 나이, 가족 정보, 표 가격 등이고 label data는 '생존 여부'`


scikitlearn의 **`train_test_split()`** 사용


- `train_test_split()` : sklearn 라이브러리의 학습용, 평가용 데이터 분리 메소드
  - `X_train, X_test, y_train, y_test = train_test_split(feature 데이터, label 데이터, test_size = 0~1 값, random_state = 랜덤 시드 값)`




## 실습: titanic 데이터 분할과 학습



In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 데이터를 읽어옵니다.


# Cabin 변수를 제거합니다.


# 결측값이 존재하는 샘플을 제거합니다.


# 이상치를 처리합니다.

# 전체 샘플 데이터 개수: 687

# 1. feature 데이터와 label 데이터를 분리합니다.

#X 데이터 개수: 687
# y 데이터 개수: 687

#2. 학습용, 평가용 데이터로 분리합니다.



# 분리한 데이터의 개수를 출력합니다.

